<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


 ## Lab 2: Data wrangling 


Estimated time needed: **60** minutes


In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models. 

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship. 

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives
Perform exploratory  Data Analysis and determine Training Labels 

- Exploratory Data Analysis
- Determine Training Labels 


----


Install the below libraries


In [1]:
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 127.7 MB/s eta 0:00:00


## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [3]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis 


Load Space X dataset, from last section.


In [4]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identify and calculate the percentage of the missing values in each attribute


In [5]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identify which columns are numerical and categorical:


In [6]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TASK 1: Calculate the number of launches on each site

The data contains several Space X  launch facilities: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Cape Canaveral Space</a> Launch Complex 40  <b>VAFB SLC 4E </b> , Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A </b>.The location of each Launch Is placed in the column <code>LaunchSite</code>


Next, let's see the number of launches for each site.

Use the method  <code>value_counts()</code> on the column <code>LaunchSite</code> to determine the number of launches  on each site: 


In [31]:
import pandas as pd
import requests

# Step 1: Get all past launches
launches = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/launches/past").json())

# Step 2: Get launchpad metadata
launchpads = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/launchpads").json())

# Step 3: Merge to get launchpad names
launches = launches.merge(launchpads[['id', 'name']], how='left', left_on='launchpad', right_on='id', suffixes=('', '_pad'))

# Step 4: Count launches by site
site_counts = launches['name'].value_counts()

# Step 5: Display result
print(site_counts)

name
FalconSat                                         1
DemoSat                                           1
Trailblazer                                       1
RatSat                                            1
RazakSat                                          1
                                                 ..
Starlink 4-20 (v1.5) & Sherpa LTC-2/Varuna-TDM    1
Starlink 4-2 (v1.5) & Blue Walker 3               1
Starlink 4-34 (v1.5)                              1
Starlink 4-35 (v1.5)                              1
Crew-5                                            1
Name: count, Length: 187, dtype: int64


In [32]:
launch_counts = launches['name_pad'].value_counts()
print(launch_counts)

name_pad
CCSFS SLC 40       99
KSC LC 39A         55
VAFB SLC 4E        28
Kwajalein Atoll     5
Name: count, dtype: int64


In [33]:
launches['name_pad'].unique()

array(['Kwajalein Atoll', 'CCSFS SLC 40', 'VAFB SLC 4E', 'KSC LC 39A'],
      dtype=object)

Each launch aims to an dedicated orbit, and here are some common orbit types:




* <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b> A geosynchronous orbit is a high Earth orbit that allows satellites to match Earth's rotation. Located at 22,236 miles (35,786 kilometers) above Earth's equator, this position is a valuable spot for monitoring weather, communications and surveillance. Because the satellite orbits at the same speed that the Earth is turning, the satellite seems to stay in place over a single longitude, though it may drift north to south,” NASA wrote on its Earth Observatory website <a  href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.
    
    
    
* <b>ES-L1 </b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.


* <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>


* <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>


* <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>


* <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

some are shown in the following plot:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TASK 2: Calculate the number and occurrence of each orbit


 Use the method  <code>.value_counts()</code> to determine the number and occurrence of each orbit in the  column <code>Orbit</code>


In [34]:
# Apply value_counts on Orbit column
import requests
import pandas as pd

# Get launches and payloads from SpaceX API
launches = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/launches/past").json())
payloads = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/payloads").json())

In [35]:
# Some launches have multiple payloads, use the first one for simplicity
launches['payload_id'] = launches['payloads'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

In [36]:
# Merge on payload ID to get the orbit column
launches = launches.merge(payloads[['id', 'orbit']], left_on='payload_id', right_on='id', how='left')

In [37]:
orbit_counts = launches['orbit'].value_counts()
print(orbit_counts)

orbit
VLEO     58
GTO      35
ISS      33
LEO      20
PO       14
SSO      12
MEO       6
TLI       2
GEO       2
ES-L1     1
HCO       1
HEO       1
SO        1
Name: count, dtype: int64


In [11]:
# Extract orbit from first payload (if available)
import requests
import pandas as pd

# Step 1: Get launches and payloads
launches = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/launches/past").json())
payloads = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/payloads").json())

# Step 2: Extract the first payload ID per launch
launches['payload_id'] = launches['payloads'].apply(lambda x: x[0] if isinstance(x, list) and x else None)

# Step 3: Merge payload orbit info
launches = launches.merge(payloads[['id', 'orbit']], left_on='payload_id', right_on='id', how='left')

# Step 4: Count orbits
orbit_counts = launches['orbit'].value_counts()
print(orbit_counts)

orbit
VLEO     58
GTO      35
ISS      33
LEO      20
PO       14
SSO      12
MEO       6
TLI       2
GEO       2
ES-L1     1
HCO       1
HEO       1
SO        1
Name: count, dtype: int64


### TASK 3: Calculate the number and occurence of mission outcome of the orbits


Use the method <code>.value_counts()</code> on the column <code>Outcome</code> to determine the number of <code>landing_outcomes</code>.Then assign it to a variable landing_outcomes.


In [38]:
launches['Outcome'] = launches['cores'].apply(
    lambda x: f"{x[0].get('landing_success')} {x[0].get('landing_type')}"
    if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict)
    else 'None None'
)

In [39]:
# Count the frequency of each outcome
landing_outcomes = launches['Outcome'].value_counts()
print(landing_outcomes)

Outcome
True ASDS      115
None None       29
True RTLS       23
False ASDS      10
True Ocean       5
False Ocean      2
None ASDS        2
False RTLS       1
Name: count, dtype: int64


In [15]:
# Combine landing_success and landing_type into a new Outcome column
df['Outcome'] = df['cores'].apply(
    lambda x: f"{x[0]['landing_success']} {x[0]['landing_type']}" 
    if isinstance(x, list) and x and isinstance(x[0], dict) and 'landing_type' in x[0] else 'None None'
)

# Count all unique outcomes
landing_outcomes = df['Outcome'].value_counts()
print(landing_outcomes)

Outcome
True ASDS      115
None None       29
True RTLS       23
False ASDS      10
True Ocean       5
False Ocean      2
None ASDS        2
False RTLS       1
Name: count, dtype: int64


<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.


In [13]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True
1 False


In [44]:
# Step 1: Filter for Falcon 9
falcon9_df = launches[launches['name'] == 'Falcon 9'].copy()

# Step 2: Compute Outcome field again on falcon9_df
falcon9_df['Outcome'] = falcon9_df['cores'].apply(
    lambda x: f"{x[0]['landing_success']} {x[0]['landing_type']}"
    if isinstance(x, list) and x and isinstance(x[0], dict)
    else "None None"
)

# Step 3: Count outcomes
outcomes = falcon9_df['Outcome'].value_counts()
print(outcomes)

# Step 4: Extract final answer
successful_drone_landings = outcomes.get('True ASDS', 0)
print("✅ Falcon 9 successful drone ship landings:", successful_drone_landings)

Series([], Name: count, dtype: int64)
✅ Falcon 9 successful drone ship landings: 0


We create a set of outcomes where the second stage did not land successfully:


In [45]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df['Outcome'].value_counts()

Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64

In [17]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

In [47]:
# Count number of launches from each Launch Site
launch_counts = df['LaunchSite'].value_counts()
print(launch_counts)

LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64


In [46]:
drone_landings = df['Outcome'].value_counts().get('True ASDS', 0)
print("Correct answer:", drone_landings)

Correct answer: 41


In [48]:
df['Orbit'].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
HEO       1
ES-L1     1
SO        1
GEO       1
Name: count, dtype: int64

### TASK 4: Create a landing outcome label from Outcome column


Using the <code>Outcome</code>,  create a list where the element is zero if the corresponding  row  in  <code>Outcome</code> is in the set <code>bad_outcome</code>; otherwise, it's one. Then assign it to the variable <code>landing_class</code>:


In [18]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
# Create landing_class: 1 = success, 0 = failure
landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]

# Optional: Add to the DataFrame for reference
df['Class'] = landing_class

# Preview
print(df[['Outcome', 'Class']].head())

     Outcome  Class
0  None None      0
1  None None      0
2  None None      0
3  None None      0
4  None None      0


This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully 


In [19]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0


In [20]:
df.head(5)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings,Orbit,Outcome,Class
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN,None,None None,0
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN,None,None None,0
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN,None,None None,0
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN,None,None None,0
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN,None,None None,0


We can use the following line of code to determine  the success rate:


In [21]:
df["Class"].mean()

np.float64(0.7647058823529411)

In [4]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 164.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 167.8 MB/s eta 0:00:00


In [5]:
import pandas as pd

In [23]:
# Filter Falcon 9 launches only
falcon9 = df[df['Version Booster'].str.contains('F9')]

# Then count successful drone ship landings
true_asds_count = falcon9[falcon9['Outcome'] == 'True ASDS'].shape[0]
print("Filtered True ASDS (Falcon 9 only):", true_asds_count)

KeyError: 'Version Booster'

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


In [8]:
import pandas as pd

# Sample dummy launch_dict with just Launch Site info (you should replace this with your actual data)
launch_dict = {
    'Flight No.': [1, 2, 3],
    'Launch Site': ['CCAFS SLC 40', 'KSC LC 39A', 'CCAFS SLC 40']
}

# Create the DataFrame
df = pd.DataFrame(launch_dict)

# Count launches from CCAFS SLC 40
ccafs_count = df[df['Launch Site'] == 'CCAFS SLC 40'].shape[0]
print("CCAFS SLC 40 launches:", ccafs_count)

CCAFS SLC 40 launches: 2


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


In [11]:
print(df.columns)

Index(['Flight No.', 'Launch Site'], dtype='object')


In [13]:
launch_dict['Launch outcome'] = []

In [14]:
# This safely grabs the outcome string from column 8 (usually index 7)
try:
    launch_outcome = list(row[7].strings)[0].strip()
except:
    launch_outcome = None

launch_dict['Launch outcome'].append(launch_outcome)

In [15]:
df['success'] = df['Launch outcome'].apply(lambda x: 'Success' in x if isinstance(x, str) else False)
success_rate = df['success'].mean() * 100
print(f"Success Rate: {success_rate:.0f}%")

KeyError: 'Launch outcome'

In [19]:
# Create Outcome column from cores
df['Outcome'] = df['cores'].apply(
    lambda x: f"{x[0]['landing_success']} {x[0]['landing_type']}"
    if isinstance(x, list) and x and 'landing_success' in x[0] and 'landing_type' in x[0]
    else 'None None'
)

In [20]:
asds_success_count = df[df['Outcome'] == 'True ASDS'].shape[0]
print("Successful drone ship landings:", asds_success_count)

Successful drone ship landings: 115


In [28]:
import pandas as pd
import requests

# Get launch data
launches = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/launches/past").json())

# Get rocket metadata
rockets = pd.json_normalize(requests.get("https://api.spacexdata.com/v4/rockets").json())

# Merge rocket names into launch data
launches = launches.merge(rockets[['id', 'name']], left_on='rocket', right_on='id', suffixes=('', '_rocket'))

# Now filter for Falcon 9
falcon9 = launches[launches['name_rocket'] == 'Falcon 9']

# Create Outcome column (if not already)
falcon9['Outcome'] = falcon9['cores'].apply(
    lambda x: f"{x[0]['landing_success']} {x[0]['landing_type']}"
    if isinstance(x, list) and x and isinstance(x[0], dict) and 'landing_success' in x[0] and 'landing_type' in x[0]
    else 'None None'
)

# Count successful drone ship landings
true_asds_count = falcon9[falcon9['Outcome'] == 'True ASDS'].shape[0]
print("✅ Successful drone ship landings (True ASDS):", true_asds_count)

✅ Successful drone ship landings (True ASDS): 114


/tmp/ipykernel_300/2150005339.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  falcon9['Outcome'] = falcon9['cores'].apply(


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
